In [1]:
import pandas as pd
import re
import csv
import numpy as np

In [55]:
files = ['filtered_'+str(n)+'_with_neighbors_normalized.csv' for n in range(9,10)]

First we create our basic commodities dataframe. Each p number is associated with a list of commodities; the list is a list of tuples (commodity name, # of commodities) for each reference of a commodity. For example if the text had said 2 lambs, 3 goats, 1 lamb, 2 sheep, the commodities list would be: [(lamb, 2), (goat, 3), (lamb, 1), (sheep, 2)]

if 2 numbers in a row, then theyre added 4(10)+3(1)
find all examples of 2 numbers in a row
if 2 nouns in a row:
goat milk: milk is an attribute of goat (goat is used for milk)
goat slaughter: goat is used for slaughter (milk attribute again)
nouns following a commodity become an attribute
and date for commodities

In [91]:
commodities = {}
p_number = ''
comms = []
comms_list = []
allowed_commodities = ['kir[lamb]N','mašgal[goat]N', 'u[ewe]N','udu[sheep]N','uzud[goat]N','sila[lamb]N',
                       'maš[goat]N','ašgar[kid]N','durah[goat]N','ab[cow]N','ud[sun]N', 'az[bear]N',
                      'gudabak[bull]N', 'šeŋbar[animal]N']
#added diš@t ban₂ barig for now for testing purposes, will get rid of once clarified
number_dict = {'barig':0, 'ban₂':0, 'diš@t':1, 'diš': 1, 'min':2,'eš':3, 'limmu':4, 'ja':5, 'aš':6, 'umun':7,'ussu':8,'ilimmu':9,'u':10, 
               'niš':20, 'ušu':30, 'nimin': 40, 'ninnu':50, 'geš₂':60, 'ĝeš-min':120, 'gešʾu':600, 'ĝes-u-min':1200, 'šar₂':3600}
for file in files:
    f = pd.read_csv(file)
    for index, row in f.iterrows():
        numbers = []
        if row['id_word'][:7] != p_number:
            commodities[p_number] = comms
            p_number = row['id_word'][:7]
            comms = []
        comm = row['lemma']
        comm_pnum = row['id_word'][:7]
        #makes sure there's a line number for some odd cases
        if row['id_word'].count('.')==2:
            comm_line = re.findall(r'\.(.*)\.',row['id_word'])[0]
            if comm in allowed_commodities:
                #grabs previous word to see if it's a number
                i = 1
                n = (f.iloc[index-i]['lemma'])
                if 'NU' in n:
                    numbers.append(n)
                    i+=1
                    preceding_word = (f.iloc[index-i]['lemma'])
                    #makes a list of preceding numbers to calculate total
                    while 'NU' in preceding_word:
                        preceding_word_line = re.findall(r'\.(.*)\.',f.iloc[index-i]['id_word'])
                        if(preceding_word_line[0] != comm_line):
                            break
                        numbers.append(preceding_word)
                        i+=1
                        preceding_word = (f.iloc[index-i]['lemma'])
                    total_for_one_commodity = 0
                    for n in numbers:
                        if n!= 'n[]NU':
                            factor = re.findall(r'(?<=\()(.*?)(?=\))',n)[0]
                            number = int(n[0])
                            integer = number_dict[factor]*number
                            total_for_one_commodity += integer
                    comms.append([comm,numbers, total_for_one_commodity])
commodities[p_number] = [comms]
del commodities['']

In [66]:
d = {'ip':list(commodities.keys()),'commodities': list(commodities.values())}
comm_df_unsplit = pd.DataFrame.from_dict(data = d)

In [67]:
comm_df_unsplit

,ip,commodities
0,P108677,"[[sila[lamb]N, [2(diš)[]NU, 1(u)[]NU], 12], [k..."
1,P107988,"[[udu[sheep]N, [1(diš)[]NU], 1], [maš[goat]N, ..."
2,P125766,"[[udu[sheep]N, [1(diš)[]NU], 1], [u[ewe]N, [1(..."
3,P131976,"[[sila[lamb]N, [6(diš)[]NU], 6]]"
4,P109756,"[[sila[lamb]N, [1(diš)[]NU], 1], [kir[lamb]N, ..."
...,...,...
1318,P130891,"[[udu[sheep]N, [5(diš)[]NU], 5], [udu[sheep]N,..."
1319,P126627,[]
1320,P103257,"[[udu[sheep]N, [8(diš)[]NU, 5(u)[]NU, 5(geš₂)[..."
1321,P142132,"[[udu[sheep]N, [4(diš)[]NU, 4(u)[]NU, 4(geš₂)[..."


Now we split up the commodities column where each list is broken up with each value having its own row.

In [88]:
comm_df = comm_df_unsplit.explode('commodities')
comm_df = comm_df.head(20)

In [89]:
comm_df['commodity'] = comm_df['commodities'].apply(lambda c: 0 if c==[] else c[0])
comm_df['numbers'] = comm_df['commodities'].apply(lambda c: 0 if c==[] else c[1])
comm_df['total'] = comm_df['commodities'].apply(lambda c: 0 if c==[] else c[2])

In [90]:
comm_df

,ip,commodities,commodity,numbers,total
0,P108677,"[sila[lamb]N, [2(diš)[]NU, 1(u)[]NU], 12]",sila[lamb]N,"[2(diš)[]NU, 1(u)[]NU]",12
0,P108677,"[kir[lamb]N, [8(diš)[]NU, 1(u)[]NU], 18]",kir[lamb]N,"[8(diš)[]NU, 1(u)[]NU]",18
0,P108677,"[ašgar[kid]N, [8(diš)[]NU, 4(u)[]NU], 48]",ašgar[kid]N,"[8(diš)[]NU, 4(u)[]NU]",48
0,P108677,"[udu[sheep]N, [3(diš)[]NU, 1(u)[]NU, 2(geš₂)[]...",udu[sheep]N,"[3(diš)[]NU, 1(u)[]NU, 2(geš₂)[]NU, 1(gešʾu)[]NU]",733
0,P108677,"[u[ewe]N, [3(diš)[]NU, 6(geš₂)[]NU], 363]",u[ewe]N,"[3(diš)[]NU, 6(geš₂)[]NU]",363
0,P108677,"[mašgal[goat]N, [6(diš)[]NU, 2(u)[]NU, 6(geš₂)...",mašgal[goat]N,"[6(diš)[]NU, 2(u)[]NU, 6(geš₂)[]NU]",386
0,P108677,"[uzud[goat]N, [8(diš)[]NU, 1(u)[]NU, 3(geš₂)[]...",uzud[goat]N,"[8(diš)[]NU, 1(u)[]NU, 3(geš₂)[]NU]",198
0,P108677,"[sila[lamb]N, [1(diš)[]NU, 2(u)[]NU, 1(geš₂)[]...",sila[lamb]N,"[1(diš)[]NU, 2(u)[]NU, 1(geš₂)[]NU]",81
0,P108677,"[kir[lamb]N, [2(diš)[]NU, 5(u)[]NU], 52]",kir[lamb]N,"[2(diš)[]NU, 5(u)[]NU]",52
0,P108677,"[maš[goat]N, [1(diš)[]NU], 1]",maš[goat]N,[1(diš)[]NU],1


In [1]:
for c in comms_list:
    comm_df["% " + str(c)] = comm_df['commodities'].map(lambda a: round(a.count(c)/len(a),3) if len(a)>0 else 0)
comm_df

NameError: name 'comms_list' is not defined

In [82]:
commodities_df.to_csv('commodities_CSV.csv')

In [ ]:
record_per_file = 10000

for j in range(len(csvfilename)):
    if j % record_per_file == 0:
        write_file = csvfilename[j:j+record_per_file]
                #adding header at the start of the write_file
                write_file.insert(0, header)
        #write in file
            open(str(fil)+ str(file) + '.csv', 'w+').writelines(write_file)
                file += 1